In [16]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.utils as utils

import gzip
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

from torchsummaryX import summary
import pytorch_warmup as warmup

from tqdm import tqdm
from sklearn.metrics import roc_auc_score

from dataloader import *
from model import *

In [17]:
path = '../iDeepS/datasets/clip/11_CLIPSEQ_ELAVL1_hg19'

instance_length = 40
instance_stride = 5

batch_size = 1
epochs = 10
lr = 5e-4
weight_decay = 1e-5

train_data_path = path + "/30000/training_sample_0/sequences.fa.gz"
valid_data_path = path + "/30000/test_sample_0/sequences.fa.gz"
train_structure_path = path + "/30000/training_sample_0/sequence_structures_forgi.out"
validate_structure_path = path + "/30000/test_sample_0/sequence_structures_forgi.out"

train_data = LibriSamplesWithStructure(train_data_path, train_structure_path)
valid_data = LibriSamplesWithStructure(valid_data_path, validate_structure_path)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = batch_size, shuffle=True)

for x, y, structure in train_loader:
    print(x.shape, y.shape, structure.shape)
    break

torch.Size([1, 13, 40, 4]) torch.Size([1, 2]) torch.Size([1, 13, 40, 6])


In [34]:
class WeakRMwithStructure(nn.Module):
    """
    used for channel = 4, AGCT
    """

    def __init__(self, training=True):
        super().__init__()

        self.inst_conv = nn.Sequential(
            nn.Conv1d(4, 32, kernel_size=15, padding=7, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.2),
            nn.Conv1d(32, 16, kernel_size=5, padding=2, stride=1),
            nn.ReLU(),
            nn.Flatten(),
        )

        self.structure_conv = nn.Sequential(
            nn.Conv1d(6, 32, kernel_size=15, padding=7, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.2),
            nn.Conv1d(32, 16, kernel_size=5, padding=2, stride=1),
            nn.ReLU(),
            nn.Flatten(),
        )

        self.attention_v = nn.Sequential(
            nn.Linear(640, 128),
            nn.Tanh()
        )

        self.attention_u = nn.Sequential(
            nn.Linear(640, 128),
            nn.Sigmoid()
        )

        self.attention_weights = nn.Sequential(
            nn.Linear(128, 1),
        )

        self.cls = nn.Sequential(
            nn.Linear(640, 2),
            nn.Sigmoid()
        )
        self.softmax = nn.Softmax()

    def forward(self, inputs, structures, training=True, mask=None):
        inputs = torch.squeeze(inputs, 0)
        inputs = inputs.permute((0, 2, 1)) 
        
        structures = torch.squeeze(structures, 0)
        structures = structures.permute((0, 2, 1)) 
        # print(structures.shape)
        inst_features = self.inst_conv(inputs) 
        structure_features = self.structure_conv(structures)
        
        # print(inst_features.shape, structure_features.shape)
        inst_features = torch.cat((inst_features, structure_features), dim = 1)
        # print(inst_features.shape)
        attention_v = self.attention_v(inst_features)
        attention_u = self.attention_v(inst_features)

        # print(attention_u*attention_v)
        # print(self.attention_weights(attention_u * attention_v))

        gated_attention = self.attention_weights(attention_u * attention_v).permute((1, 0))
        # print(gated_attention)

        gated_attention = self.softmax(gated_attention)  # torch.Size([1, 13])

        bag_features = torch.matmul(gated_attention, inst_features)

        bag_probability = self.cls(bag_features)

        return bag_probability, gated_attention


In [37]:
model = WeakRMwithStructure().cuda()
# model = WSCNN().cuda()

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.005)
criterion = nn.BCELoss(weight=torch.tensor([0.8, 0.2])).cuda()
num_steps = len(train_loader) * epochs
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_steps)
warmup_scheduler = warmup.RAdamWarmup(optimizer)


for epoch in range(1, epochs + 1):
        # training
        model.train()
        num_correct = 0
        total_loss = 0
        for i, (x, y, structure) in enumerate(train_loader):
            # print(i)
            optimizer.zero_grad()

            x = x.float().cuda()
            y = y.float().cuda()
            structure = structure.float().cuda()
            # print(structure[:, :13, :, :].shape, structure.shape, "!")
            outputs, _ = model(x, structure[:, :13, :, :])

            loss = criterion(outputs[0], y[0])

            outputs = torch.argmax(outputs)

            num_correct += int((outputs == torch.argmax(y)).sum())
            total_loss += loss

            loss.backward()
            optimizer.step()
            with warmup_scheduler.dampening():
                scheduler.step()

        print_content ="Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(
            epoch,
            epochs,
            100 * num_correct / (len(train_loader) * batch_size),
            float(total_loss / len(train_loader)),
            float(optimizer.param_groups[0]['lr'])
        )

#         print(print_content)
        train_acc =  100 * num_correct / (len(train_loader) * batch_size)
        train_loss = float(total_loss / len(train_loader))
        learn_rate = float(optimizer.param_groups[0]['lr'])

        # eval
        model.eval()

        num_correct = 0
        total_loss = 0
        predictions =[]
        labels = []
        for i, (x, y, structure) in enumerate(valid_loader):
            x = x.float().cuda()
            y = y.float().cuda()
            structure = structure.float().cuda()
            
            with torch.no_grad():
                outputs_probs, _ = model(x, structure[:, :13, :, :])

            outputs = torch.argmax(outputs_probs)

            num_correct += int((outputs == torch.argmax(y)).sum())
            total_loss += loss

            predictions.append(outputs_probs.detach().cpu().numpy()[0])
            labels.append(torch.argmax(y).detach().cpu().numpy())

        dev_acc = 100 * num_correct / len(valid_loader)
        dev_loss = total_loss / len(valid_loader)

        auc_score = roc_auc_score(np.array(labels).flatten(), np.array(predictions)[:, 1])
        print("Validation Accuracy {:.04f}%, Auc Score {:.04f}%".format(dev_acc, auc_score))


/tmp/ipykernel_2270515/1773639120.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  gated_attention = self.softmax(gated_attention)  # torch.Size([1, 13])


Validation Accuracy 84.9400%, Auc Score 0.9035%


/tmp/ipykernel_2270515/1773639120.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  gated_attention = self.softmax(gated_attention)  # torch.Size([1, 13])


Validation Accuracy 84.1700%, Auc Score 0.9002%


/tmp/ipykernel_2270515/1773639120.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  gated_attention = self.softmax(gated_attention)  # torch.Size([1, 13])


...

Validation Accuracy 84.8900%, Auc Score 0.9030%


/tmp/ipykernel_2270515/1773639120.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  gated_attention = self.softmax(gated_attention)  # torch.Size([1, 13])


Validation Accuracy 86.8700%, Auc Score 0.9049%


/tmp/ipykernel_2270515/1773639120.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  gated_attention = self.softmax(gated_attention)  # torch.Size([1, 13])


KeyboardInterrupt: 

In [ ]:
[1, 2, 3, 4][:3]

In [27]:
structures

NameError: name 'structures' is not defined